## This laboratory will be conducted over two sessions
## You are expected to finish the software and test the code in simulation between the sessions! ##

# Manipulating objects

For this laboratory, we will use two robots distant from each other of 25cm as depicted in the figure below

<img src="./2_robot_config.png" width="500">

There is 1cm between the end-effector "shell" and the position of the frame as depicted in the figure below

<img src="./foot_close.png" width="300">

<img src="./box.jpg" width="200">

The objects to grasp are soft cubes with a 5cm side.
Three cubes are stacked on top of each other. The goal of the laboratory is to design a controller to will move the two cubes on the top away
and pick the bottom cube and lift it up by 10cm.

In [1]:
#setup nice plotting (use widget instead of notebook in the command below if you use jupyter lab)
%matplotlib widget

# we import useful libraries
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

# this function returns the body Jacobian
from robot_utils import getBodyJacobianOrientedLikeSpatialFrame
from scipy import optimize

# we import the robot class
use_real_robot = False

if use_real_robot:
    from nyu_finger import NYUFingerReal    
else:
    from nyu_finger_simulator import NYUFingerSimulator


# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

pybullet build time: Dec  1 2021 18:33:43


In [2]:
def translate(vector):
    transform = np.eye(4)
    transform[:3, 3] = vector
    return transform

def rotateX(angle):
    transform = np.eye(4)
    sin = np.sin(angle)
    cos = np.cos(angle)
    transform[1, 1] = cos
    transform[1, 2] = -sin
    transform[2, 1] = sin
    transform[2, 2] = cos
    return transform

def rotateY(angle):
    transform = np.eye(4)
    sin = np.sin(angle)
    cos = np.cos(angle)
    transform[0, 0] = cos
    transform[0, 2] = sin
    transform[2, 0] = -sin
    transform[2, 2] = cos
    return transform

def rotateZ(angle):
    transform = np.eye(4)
    sin = np.sin(angle)
    cos = np.cos(angle)
    transform[0, 0] = cos
    transform[0, 1] = -sin
    transform[1, 0] = sin
    transform[1, 1] = cos
    return transform


def forward_kinematics(theta):
    # assume theta is a vector, theta[0] = theta0, theta[1] = theta1, etc.
    # we define the lengths of the robot
    l0 = 0.3
    l1 = 0.16
    l2 = 0.16
    l3 = 0.014
    
    ###WRITE YOUR CODE HERE
    T = np.eye(4)
    
    TSH1 = translate(np.array([l0, 0, 0])) @ rotateX(theta[0]) 
    
    TSH2 = TSH1 @ translate(np.array([0, 0, l3])) @ rotateZ(theta[1]) 

    TSK = TSH2 @ translate(np.array([0, -l1, 0])) @ rotateZ(theta[2])
    
    TSF = TSK @ translate(np.array([0, -l2, 0]))
    
    ### we return the object
    return TSF

In [3]:
def linear_interpolate_trajectory(position_init, position_goal, t_init, t_goal, t):
    s = (t - t_init)/(t_goal - t_init)
    desired_position = position_init + s * (position_goal - position_init)
    desired_velocity = 1/(t_goal - t_init) * (position_goal - position_init)
    
    # we return the answer
    return desired_position, desired_velocity

def second_order_trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    s = 3/(T**2)*((t-t_init)**2) - 2/ (T**3)*(t-t_init)**3
    desired_position = position_init + s * (position_goal - position_init)
    desired_velocity = (6/(T**2)*(t-t_init) - 6/(T**3)*(t-t_init)**2) * (position_goal - position_init)
    
    # we return the answer
    return desired_position, desired_velocity

def third_order_trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    desired_position = position_init + (10/(T**3)*(t-t_init)**3 + -15/(T**4)*(t-t_init)**4 + \
                                        6/(T**5)*(t-t_init)**5)*(position_goal-position_init)
    desired_velocity = (30/(T**3)*(t-t_init)**2+-60/(T**4)*(t-t_init)**3+30/(T**5)*(t-t_init)**4)*(position_goal-position_init)
    
    # we return the answer
    return desired_position, desired_velocity

In [4]:
def fingertip_position_error(joint_angles, desired_fingertip_position):
    """
    fingertip_position_error function
    input joint_angles (as a 3D vector) and desired_fingertip_position (as a 3D vector)
    output the squared distance between the desired fingertip position and the one realized by joint_angles
    """
    p_fk = forward_kinematics(joint_angles)
    error = np.subtract(p_fk[:3, 3], desired_fingertip_position)
    return error.dot(error)


def inverse_geometry(desired_fingertip_position):
    """
    inverse geometry function
    input desired_fingertip_position (3D vector of desired position of the fingertip)
    output a vector of joint angles
    """
    res = optimize.minimize(fingertip_position_error, np.array([0.0, 0.0, 0.0]), args = (desired_fingertip_position))
    return res.x

## Questions
In this laboratory you are expected to re-use code from the previous laboratories (e.g. to plot, compute forward kinematics, etc).

1. Using an impedance controller (Lab4 - use the ``getBodyJacobianOrientedLikeSpatialFrame`` function to get the Jacobian) and one trajectory generation method seen in Lab3 write a complete controller that solves the problem: move the top two cubes away, grasp the last cube and move it up by 10cm.

2. Test your controller in simulation. Save and plot the results.
   
3. Test the controller on the real robots. Save and plot the results.

4. In the report, describe in details the controller you designed (what it does, etc) and the results you obtained.

5. (optional - bonus) Extend the controller to rearrange the cubes - putting the bottom one on top of the two others.

In [5]:
pos_1 = [0.44, -.16, 0]

In [ ]:
# we create a robot simulation or a real robot
if use_real_robot:
    robot1 = NYUFingerReal()
    motor_number = np.array([1,2,3])
    robot1.initialize('enp5s0f1', motor_number)
    
    robot2 = NYUFingerReal()
    motor_number = np.array([1,2,3])
    robot2.initialize('eno1', motor_number)

else:
    robot1 = NYUFingerSimulator(robotStartPos = [0.,0.,0.])
    robot1.add_box(position=[0.15, -0.05, 0.01], friction=0.5)
    robot1.add_box(position=[0.15, -0.05, 0.07], friction=0.5)
    robot1.add_box(position=[0.15, -0.05, 0.13], friction=0.5)
    # we put the 2nd robot 0.3m away
    robot2 = NYUFingerSimulator(robotStartPos = [0.3,0.,0.])
    # we reset the simulation to the initial position we want to move
    robot1.reset_state(np.array([0.0,0.0,0.5]))
    robot2.reset_state(np.array([0.0,0.0,-0.5]))

# we reset the simulation
if use_real_robot:
    # we wait
    for i in range(7500):
        robot1.send_joint_torque(np.zeros([3,1]))
        robot2.send_joint_torque(np.zeros([3,1]))
        robot1.step()

# we simulate for 10 seconds
run_time = 13.
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([1.5, 1.5, 1.5])
D = np.array([0.05, 0.05, 0.05])


## we allocate a few vectors to collect data while we do the simulation
# we keep track of time
time = np.zeros([num_steps])
# we will store the sensor values (angular position and velocities)
measured_positions_f1 = np.zeros([num_steps,3])
measured_velocities_f1 = np.zeros_like(measured_positions_f1)
measured_positions_f2 = np.zeros_like(measured_positions_f1)
measured_velocities_f2 = np.zeros_like(measured_positions_f1)
# we will store the end effector position (desired and measured)
measured_finger_position_f1 = np.zeros([num_steps,3])
desired_finger_position_f1 = np.zeros([num_steps,3])
measured_finger_position_f2 = np.zeros([num_steps,3])
desired_finger_position_f2 = np.zeros([num_steps,3])

time_to_goal = 1.5
block_number = 0
t = 0


for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q1, dq1 = robot1.get_state()
    q2, dq2 = robot2.get_state()
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER

    joint_torques1 = np.zeros([3,1])
    joint_torques2 = np.zeros([3,1])
    
    pose1 = forward_kinematics(q1)
    measured_finger_position_f1[i,:] = pose1[0:3,3]
    print(measured_finger_position_f1[i,:])
    
    if(time[i] == 0):
        q_mog, pos_mog, init_time_mog, time_to_mog = \
        (measured_finger_position_f1[i,:], pos_1, 0, time_to_goal)
        
    pos_des, vel_des = second_order_trajectory(q_mog, \
                        pos_mog, init_time_mog, time_to_mog, time[i])

    
    # we send them to the robot and do one simulation step
    J1 = getBodyJacobianOrientedLikeSpatialFrame(q1);
    
    K = np.array([[10, 0, 0],
                  [0, 10, 0],
                  [0, 0, 10]])
    
    D = np.array([[.5, 0, 0],
                  [0, .5, 0],
                  [0, 0, .5]])
    
    desired_finger_position_f1[i, :] = pos_des
    desired_velocities_f1= vel_des
    
    joint_torques1 = J1.T @ (K @ (desired_finger_position_f1[i, :] - measured_finger_position_f1[i,:]) + \
                  D @ (desired_velocities_f1 - (J1 @ dq1)))
        
    # we send them to the robot and do one simulation step
    robot1.send_joint_torque(joint_torques1)
    #robot2.send_joint_torque(joint_torques2)
    robot1.step()

[ 0.37670809 -0.30041321  0.014     ]
[ 0.37670412 -0.30041605  0.01400457]
[ 0.37669629 -0.30042169  0.01401368]
[ 0.37668467 -0.30043006  0.01402727]
[ 0.37666936 -0.30044111  0.01404532]
[ 0.37665044 -0.3004548   0.01406776]
[ 0.37662802 -0.30047106  0.01409458]
[ 0.37660217 -0.30048983  0.01412571]
[ 0.37657299 -0.30051107  0.01416112]
[ 0.37654056 -0.30053472  0.01420076]
[ 0.37650497 -0.30056071  0.01424458]
[ 0.37646632 -0.30058899  0.01429255]
[ 0.37642468 -0.3006195   0.01434461]
[ 0.37638014 -0.30065219  0.01440071]
[ 0.37633279 -0.30068699  0.01446082]
[ 0.37628272 -0.30072385  0.01452489]
[ 0.37623    -0.3007627   0.01459286]
[ 0.37617473 -0.30080349  0.01466468]
[ 0.37611699 -0.30084615  0.01474032]
[ 0.37605686 -0.30089064  0.01481971]
[ 0.37599442 -0.30093688  0.01490281]
[ 0.37592976 -0.30098482  0.01498958]
[ 0.37586295 -0.30103441  0.01507995]
[ 0.37579409 -0.30108557  0.01517388]
[ 0.37572324 -0.30113826  0.01527131]
[ 0.37565048 -0.30119241  0.0153722 ]
[ 0.37557591

[ 0.37899889 -0.30509172  0.03783726]
[ 0.37907876 -0.30506012  0.03776812]
[ 0.37915829 -0.30502821  0.03769866]
[ 0.37923747 -0.30499599  0.03762892]
[ 0.37931631 -0.30496346  0.0375589 ]
[ 0.37939481 -0.30493061  0.03748862]
[ 0.37947295 -0.30489744  0.03741812]
[ 0.37955074 -0.30486394  0.03734739]
[ 0.37962818 -0.30483011  0.03727648]
[ 0.37970527 -0.30479594  0.03720539]
[ 0.37978199 -0.30476144  0.03713414]
[ 0.37985836 -0.30472658  0.03706275]
[ 0.37993437 -0.30469138  0.03699125]
[ 0.38001002 -0.30465582  0.03691964]
[ 0.38008531 -0.30461991  0.03684796]
[ 0.38016023 -0.30458363  0.03677621]
[ 0.38023479 -0.30454698  0.03670442]
[ 0.38030899 -0.30450996  0.0366326 ]
[ 0.38038282 -0.30447256  0.03656076]
[ 0.38045629 -0.30443479  0.03648894]
[ 0.38052939 -0.30439662  0.03641715]
[ 0.38060213 -0.30435806  0.03634539]
[ 0.3806745  -0.30431911  0.03627369]
[ 0.3807465  -0.30427975  0.03620208]
[ 0.38081814 -0.30424     0.03613055]
[ 0.38088941 -0.30419983  0.03605913]
[ 0.38096032

[ 0.39316243 -0.28067271  0.03254418]
[ 0.39322755 -0.28055963  0.0325568 ]
[ 0.39329312 -0.28044682  0.03256926]
[ 0.39335913 -0.28033427  0.03258158]
[ 0.3934256  -0.28022198  0.03259373]
[ 0.39349251 -0.28010997  0.03260574]
[ 0.39355989 -0.27999822  0.03261758]
[ 0.39362772 -0.27988675  0.03262927]
[ 0.393696   -0.27977554  0.0326408 ]
[ 0.39376475 -0.2796646   0.03265216]
[ 0.39383396 -0.27955394  0.03266336]
[ 0.39390364 -0.27944355  0.0326744 ]
[ 0.39397377 -0.27933343  0.03268527]
[ 0.39404438 -0.27922359  0.03269598]
[ 0.39411545 -0.27911402  0.03270652]
[ 0.39418699 -0.27900472  0.03271689]
[ 0.39425901 -0.2788957   0.03272709]
[ 0.39433149 -0.27878695  0.03273712]
[ 0.39440444 -0.27867847  0.03274698]
[ 0.39447787 -0.27857026  0.03275667]
[ 0.39455177 -0.27846233  0.03276618]
[ 0.39462615 -0.27835467  0.03277553]
[ 0.394701   -0.27824729  0.0327847 ]
[ 0.39477633 -0.27814017  0.03279369]
[ 0.39485214 -0.27803332  0.03280251]
[ 0.39492842 -0.27792675  0.03281116]
[ 0.39500517

[ 0.41466685 -0.25771914  0.032342  ]
[ 0.41466652 -0.25766644  0.03233817]
[ 0.41466619 -0.25761274  0.03233426]
[ 0.41466585 -0.25755802  0.03233029]
[ 0.41466551 -0.25750228  0.03232624]
[ 0.41466516 -0.25744554  0.03232212]
[ 0.41466481 -0.25738778  0.03231793]
[ 0.41466446 -0.25732902  0.03231366]
[ 0.4146641  -0.25726924  0.03230932]
[ 0.41466374 -0.25720845  0.03230491]
[ 0.41466337 -0.25714666  0.03230043]
[ 0.414663   -0.25708386  0.03229587]
[ 0.41466263 -0.25702005  0.03229125]
[ 0.41466225 -0.25695524  0.03228655]
[ 0.41466187 -0.25688942  0.03228178]
[ 0.41466149 -0.2568226   0.03227694]
[ 0.4146611  -0.25675479  0.03227202]
[ 0.41466069 -0.256686    0.03226704]
[ 0.4146603  -0.25661619  0.03226199]
[ 0.4146599  -0.25654539  0.03225686]
[ 0.4146595  -0.2564736   0.03225166]
[ 0.4146591  -0.25640081  0.03224639]
[ 0.4146587  -0.25632705  0.03224106]
[ 0.41465829 -0.25625229  0.03223565]
[ 0.41465789 -0.25617656  0.03223017]
[ 0.41465748 -0.25609985  0.03222462]
[ 0.41465707

[ 0.41473228 -0.22842449  0.03022083]
[ 0.41472297 -0.22837017  0.03021796]
[ 0.41471417 -0.22831706  0.03021584]
[ 0.41470592 -0.22826517  0.03021431]
[ 0.41469821 -0.22821453  0.03021321]
[ 0.41469104 -0.22816517  0.0302124 ]
[ 0.41468441 -0.2281171   0.03021179]
[ 0.41467833 -0.22807033  0.0302113 ]
[ 0.4146728  -0.22802485  0.03021089]
[ 0.41466781 -0.22798065  0.03021051]
[ 0.4146634  -0.2279377   0.03021013]
[ 0.41466571 -0.22788556  0.03021055]
[ 0.41467462 -0.22782441  0.03021174]
[ 0.41469003 -0.22775445  0.03021368]
[ 0.41471184 -0.22767586  0.03021634]
[ 0.41473994 -0.22758884  0.03021971]
[ 0.41474586 -0.22753353  0.03022165]
[ 0.41474971 -0.227482    0.03022186]
[ 0.41475287 -0.22743226  0.03022057]
[ 0.41475537 -0.22738413  0.03021822]
[ 0.4147572  -0.2273374   0.03021533]
[ 0.41475852 -0.22729188  0.03021194]
[ 0.41475937 -0.22724741  0.03020825]
[ 0.41475981 -0.22720385  0.03020447]
[ 0.41476002 -0.22716092  0.03020063]
[ 0.41476013 -0.22711844  0.03019674]
[ 0.41476119

...

[ 0.18117842 -0.29595395  0.02979397]
[ 0.18116825 -0.29594996  0.02979303]
[ 0.18115809 -0.29594597  0.02979209]
[ 0.18114794 -0.295942    0.02979116]
[ 0.18113781 -0.29593803  0.02979024]
[ 0.18112771 -0.29593406  0.02978932]
[ 0.18111762 -0.29593011  0.0297884 ]
[ 0.18110755 -0.29592616  0.02978749]
[ 0.18109751 -0.29592222  0.0297866 ]
[ 0.1810875  -0.29591828  0.02978571]
[ 0.18107751 -0.29591436  0.02978483]
[ 0.18106754 -0.29591045  0.02978396]
[ 0.18105761 -0.29590654  0.02978311]
[ 0.1810477  -0.29590265  0.02978227]
[ 0.18103782 -0.29589876  0.02978144]
[ 0.18102798 -0.29589489  0.02978062]
[ 0.18101816 -0.29589103  0.02977983]
[ 0.18100838 -0.29588718  0.02977904]
[ 0.18099863 -0.29588334  0.02977827]
[ 0.18098892 -0.29587951  0.02977752]
[ 0.18097924 -0.29587569  0.02977679]
[ 0.1809696  -0.29587188  0.02977607]
[ 0.18095998 -0.29586809  0.02977537]
[ 0.18095041 -0.29586431  0.02977469]
[ 0.18094087 -0.29586054  0.02977402]
[ 0.18093137 -0.29585678  0.02977337]
[ 0.1809219 

[ 0.17878928 -0.2949987   0.0296488 ]
[ 0.17878212 -0.2949958   0.02964844]
[ 0.17877495 -0.29499289  0.02964807]
[ 0.17876779 -0.29498999  0.02964769]
[ 0.17876062 -0.29498708  0.0296473 ]
[ 0.17875345 -0.29498418  0.02964691]
[ 0.17874627 -0.29498127  0.02964651]
[ 0.17873909 -0.29497836  0.02964611]
[ 0.17873191 -0.29497545  0.02964569]
[ 0.17872473 -0.29497254  0.02964527]
[ 0.17871754 -0.29496963  0.02964484]
[ 0.17871035 -0.29496672  0.02964441]
[ 0.17870316 -0.29496381  0.02964397]
[ 0.17869597 -0.2949609   0.02964352]
[ 0.17868877 -0.29495799  0.02964306]
[ 0.17868157 -0.29495507  0.02964259]
[ 0.17867437 -0.29495216  0.02964212]
[ 0.17866717 -0.29494925  0.02964164]
[ 0.17865997 -0.29494634  0.02964116]
[ 0.17865277 -0.29494342  0.02964066]
[ 0.17864556 -0.29494051  0.02964017]
[ 0.17863836 -0.2949376   0.02963966]
[ 0.17863115 -0.29493469  0.02963915]
[ 0.17862395 -0.29493178  0.02963864]
[ 0.17861675 -0.29492887  0.02963811]
[ 0.17860955 -0.29492596  0.02963759]
[ 0.17860236

[ 0.17703061 -0.29428154  0.0295371 ]
[ 0.17702509 -0.29427927  0.02953679]
[ 0.17701958 -0.294277    0.02953648]
[ 0.17701408 -0.29427473  0.02953618]
[ 0.17700857 -0.29427246  0.02953587]
[ 0.17700308 -0.2942702   0.02953557]
[ 0.17699758 -0.29426793  0.02953526]
[ 0.17699209 -0.29426567  0.02953495]
[ 0.1769866  -0.29426341  0.02953464]
[ 0.17698112 -0.29426115  0.02953433]
[ 0.17697564 -0.29425889  0.02953402]
[ 0.17697016 -0.29425663  0.02953371]
[ 0.17696468 -0.29425437  0.02953339]
[ 0.1769592  -0.29425211  0.02953308]
[ 0.17695373 -0.29424986  0.02953276]
[ 0.17694826 -0.2942476   0.02953244]
[ 0.17694279 -0.29424535  0.02953211]
[ 0.17693732 -0.2942431   0.02953179]
[ 0.17693186 -0.29424084  0.02953146]
[ 0.17692639 -0.29423859  0.02953113]
[ 0.17692093 -0.29423634  0.02953079]
[ 0.17691547 -0.29423409  0.02953046]
[ 0.17691001 -0.29423184  0.02953012]
[ 0.17690455 -0.29422959  0.02952978]
[ 0.17689909 -0.29422735  0.02952943]
[ 0.17689364 -0.2942251   0.02952908]
[ 0.17688818

[ 0.17567303 -0.29371951  0.0294505 ]
[ 0.17566868 -0.2937177   0.02945023]
[ 0.17566434 -0.29371588  0.02944996]
[ 0.17566    -0.29371408  0.02944969]
[ 0.17565566 -0.29371227  0.02944942]
[ 0.17565133 -0.29371046  0.02944915]
[ 0.175647   -0.29370865  0.02944888]
[ 0.17564266 -0.29370685  0.02944861]
[ 0.17563834 -0.29370504  0.02944833]
[ 0.17563401 -0.29370324  0.02944806]
[ 0.17562969 -0.29370144  0.02944779]
[ 0.17562537 -0.29369964  0.02944751]
[ 0.17562105 -0.29369783  0.02944723]
[ 0.17561673 -0.29369604  0.02944695]
[ 0.17561242 -0.29369424  0.02944668]
[ 0.1756081  -0.29369244  0.0294464 ]
[ 0.17560379 -0.29369064  0.02944612]
[ 0.17559949 -0.29368885  0.02944583]
[ 0.17559518 -0.29368705  0.02944555]
[ 0.17559088 -0.29368526  0.02944527]
[ 0.17558658 -0.29368347  0.02944498]
[ 0.17558228 -0.29368168  0.0294447 ]
[ 0.17557799 -0.29367989  0.02944441]
[ 0.1755737  -0.2936781   0.02944413]
[ 0.17556941 -0.29367631  0.02944384]
[ 0.17556512 -0.29367452  0.02944356]
[ 0.17556084

[ 0.17459233 -0.29326692  0.02938024]
[ 0.17458887 -0.29326547  0.02938   ]
[ 0.17458541 -0.29326401  0.02937977]
[ 0.17458196 -0.29326256  0.02937954]
[ 0.17457851 -0.29326111  0.02937931]
[ 0.17457506 -0.29325965  0.02937908]
[ 0.17457161 -0.2932582   0.02937885]
[ 0.17456817 -0.29325676  0.02937862]
[ 0.17456473 -0.29325531  0.0293784 ]
[ 0.1745613  -0.29325386  0.02937817]
[ 0.17455787 -0.29325242  0.02937794]
[ 0.17455444 -0.29325097  0.02937771]
[ 0.17455101 -0.29324953  0.02937749]
[ 0.17454759 -0.29324809  0.02937726]
[ 0.17454417 -0.29324665  0.02937704]
[ 0.17454076 -0.29324521  0.02937681]
[ 0.17453734 -0.29324378  0.02937659]
[ 0.17453393 -0.29324234  0.02937636]
[ 0.17453053 -0.29324091  0.02937614]
[ 0.17452713 -0.29323947  0.02937592]
[ 0.17452373 -0.29323804  0.0293757 ]
[ 0.17452033 -0.29323661  0.02937548]
[ 0.17451694 -0.29323518  0.02937526]
[ 0.17451355 -0.29323375  0.02937504]
[ 0.17451016 -0.29323233  0.02937482]
[ 0.17450678 -0.2932309   0.0293746 ]
[ 0.1745034 